In [1]:
# === Simulation d’un dataset santé simplifié 
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

np.random.seed(42)
n = 1000

# Variables explicatives
age = np.random.randint(30, 65, size=n)
gender = np.random.choice([0, 1], size=n)  # 0 = femme, 1 = homme
smoker = np.random.choice([0, 1], size=n, p=[0.9, 0.1])  # déséquilibré
alcohol = np.random.choice([0, 1], size=n, p=[0.92, 0.08])  # déséquilibré

# Variable cible équilibrée
cardio_risk = np.random.choice([0, 1], size=n, p=[0.5, 0.5])

# Dataset initial équilibré
df = pd.DataFrame({
    'age': age,
    'gender': gender,
    'smoker': smoker,
    'alcohol': alcohol,
    'cardio_risk': cardio_risk
})

In [2]:
# === FEATURE ENGINEERING : on crée une nouvelle variable combinée ===
df['risk_behavior'] = df['smoker'] + df['alcohol']

# === Cas 1 : variable cible ÉQUILIBRÉE ===
X_eq = df[['age', 'gender', 'smoker', 'alcohol', 'risk_behavior']]
y_eq = df['cardio_risk']

# Régression sans pénalisation
model_none = LogisticRegression(max_iter=1000)
model_none.fit(X_eq, y_eq)
print("=== Coefficients (pas de pénalisation, cible équilibrée) ===")
print(dict(zip(X_eq.columns, model_none.coef_[0])))


=== Coefficients (pas de pénalisation, cible équilibrée) ===
{'age': np.float64(-0.0032711489641516402), 'gender': np.float64(-0.05790551419579475), 'smoker': np.float64(-0.04201396439298932), 'alcohol': np.float64(-0.00989888762265239), 'risk_behavior': np.float64(-0.05191285201564878)}


In [ ]:

# Régression avec pénalisation L2 RIDGE
model_l2 = LogisticRegression(penalty='l2', C=1.0, max_iter=1000)
model_l2.fit(X_eq, y_eq)
print("=== Coefficients (L2 - Ridge) ===")
print(dict(zip(X_eq.columns, model_l2.coef_[0])))

=== Coefficients (L2 - Ridge) ===
{'age': np.float64(-0.0032711489641516402), 'gender': np.float64(-0.05790551419579475), 'smoker': np.float64(-0.04201396439298932), 'alcohol': np.float64(-0.00989888762265239), 'risk_behavior': np.float64(-0.05191285201564878)}


In [ ]:

# Régression avec pénalisation - L1 LASSO
model_l1 = LogisticRegression(penalty='l1', solver='liblinear', C=1.0, max_iter=1000)
model_l1.fit(X_eq, y_eq)
print("=== Coefficients (L1 - Lasso) ===")
print(dict(zip(X_eq.columns, model_l1.coef_[0])))

=== Coefficients (L1 - Lasso) ===
{'age': np.float64(-0.0014396361973545858), 'gender': np.float64(-0.033703346582114914), 'smoker': np.float64(0.0), 'alcohol': np.float64(0.0), 'risk_behavior': np.float64(-0.049116945776722404)}


In [5]:

# QUESTION : Que fait L1 ici ?


# RÉPONSE :
# La pénalisation L1 pousse certains coefficients vers zéro → sélection automatique de variables pertinentes.
# Utile quand on a trop de variables ou du bruit dans les données.




# === Cas 2 : variable cible DÉSÉQUILIBRÉE (90% non à risque) ===
# Simulation d’un déséquilibre
df_imb = pd.concat([
    df[df['cardio_risk'] == 0].sample(frac=0.9, random_state=1),
    df[df['cardio_risk'] == 1].sample(frac=0.1, random_state=1)
])

X_imb = df_imb[['age', 'gender', 'smoker', 'alcohol', 'risk_behavior']]
y_imb = df_imb['cardio_risk']

# Régression sans class_weight → biaisé par la classe majoritaire
model_imb_naive = LogisticRegression(max_iter=1000)
model_imb_naive.fit(X_imb, y_imb)
print("=== Déséquilibré, sans class_weight ===")
print(dict(zip(X_imb.columns, model_imb_naive.coef_[0])))
print(classification_report(y_imb, model_imb_naive.predict(X_imb)))

# Régression avec class_weight='balanced' → corrige le déséquilibre
model_balanced = LogisticRegression(class_weight='balanced', max_iter=1000)
model_balanced.fit(X_imb, y_imb)
print("=== Déséquilibré, avec class_weight='balanced' ===")
print(dict(zip(X_imb.columns, model_balanced.coef_[0])))
print(classification_report(y_imb, model_balanced.predict(X_imb)))

# QUESTION : Pourquoi le modèle naïf est-il mauvais ?
# RÉPONSE :
# Il apprend surtout à prédire la classe majoritaire (0).
# Il a une précision globale élevée, mais ignore la minorité (1).

# QUESTION : Que fait class_weight='balanced' ?
# RÉPONSE :
# Il donne plus de poids aux erreurs sur la classe minoritaire pour forcer l’algorithme à en tenir compte.
# Cela permet de mieux détecter les cas de risque malgré leur rareté.

# QUESTION : Est-ce utile d’utiliser L1 ou L2 en plus ?
# RÉPONSE :
# Oui, on peut combiner penalty='l1' ou 'l2' avec class_weight='balanced' pour faire à la fois :
# - pondération des classes
# - sélection ou régularisation des variables

# === Conclusion pour les apprenants ===
# - Toujours inspecter l’équilibre des classes cible.
# - Faire du feature engineering métier (ex : risk_behavior) améliore les performances.
# - Adapter le modèle (class_weight, pénalisation) selon la distribution des données.

=== Déséquilibré, sans class_weight ===
{'age': np.float64(-0.011238955577746975), 'gender': np.float64(0.021319132801369444), 'smoker': np.float64(0.20246917605166942), 'alcohol': np.float64(-0.35676354476154065), 'risk_behavior': np.float64(-0.15429436870983232)}
              precision    recall  f1-score   support

           0       0.90      1.00      0.95       461
           1       0.00      0.00      0.00        49

    accuracy                           0.90       510
   macro avg       0.45      0.50      0.47       510
weighted avg       0.82      0.90      0.86       510

=== Déséquilibré, avec class_weight='balanced' ===
{'age': np.float64(-0.010997601319743802), 'gender': np.float64(0.034192448143937314), 'smoker': np.float64(0.24333444308963734), 'alcohol': np.float64(-0.4406575716883497), 'risk_behavior': np.float64(-0.19732312859872583)}
              precision    recall  f1-score   support

           0       0.90      0.47      0.62       461
           1       0.1

c:\Users\coule\Documents\projets\cardio-vasc-risks\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\coule\Documents\projets\cardio-vasc-risks\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\coule\Documents\projets\cardio-vasc-risks\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(avera